In [14]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"   

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.76 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.
google-auth 2.22.0 requires urllib3<2.0, but you have urllib3 2.2.1 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2024.3.1 which is incompatible.
spacy 3.6.0 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
tensorflow-gpu 2.4.0 requires absl-py~=0.10, but you have absl-py 1.4.0 which is incompatible.
tensorflow-gpu 2.4.0 requires flatbuffers~=1.12.0, but you have flatbuffers 2.0.7 which is incompatible.
tensorflow-gpu 2.4.0 requires grpcio~=1.32.0, but you have grpcio 1.64.1 which is incompatible.
tensorflow-gpu 2.4.0 requires h5py~=2.10.0

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.25 requires torch==2.2.1, but you have torch 2.0.1 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [2]:
local_path = "Hashim_maniyar_resume.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [3]:
data[0].page_content

'Hashim Maniyar +91-9741716323 |\n\nsamimaniyar2003@gmail.com|\n\nLinkedIn |\n\nGitHub Account\n\nEDUCATION\n\n\n\nPES University, Bangalore Bachelor of Technology in Computer Science and Engineering; GPA: 8.0/10.0\n\nBangalore Sep. 2021 – June 2025\n\nRELEVANT COURSES AND SKILLS\n\nCourses: NLP, AI, CV, ML, Reinforcement Learning, Large Scale ML, Computer Vision & DSA\n\nTech Skills: C, C++, Git, Java, Python, R, PyTorch, Sklearn, Tensorflow, spaCy, SQL, Data Analytics, React.JS, MERN & Flask\n\nAchievements\n\nRanked in the top 2% leaderboard of a hackathon conducted across more than 200 teams. Nov ’23\n\nCertifications\n\nHackerRank Certification(Basic)\n\nHackerRank Certification(Intermediate)\n\nCoursera Machine Learning Course by Andrew Ng:\n\nPart 1 - Supervised Machine Learning: Regression and Classification\n\nPart 2 - Advanced Learning Algorithms\n\nCOURSE PROJECTS\n\nSentiment Analysis\n\nConducted sentiment analysis on movie reviews using natural language processing techniq

In [4]:
!ollama pull nomic-embed-text

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90...   0% ▕                ▏    0 B/274 MB                  pulling manifest 
pulling 970aa74c0a90... 

In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [8]:
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 1/1 [00:09<00:00,  9.37s/it]


In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [10]:
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [11]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [12]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
!ollama pull mistral 

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB                  pulling manifest 
pulling ff82381e2bea...   0% ▕                ▏    0 B/4.1 GB 

In [16]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:05<00:00,  5.79s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.10s/it]
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1


' The name of the applicant is Hashim Maniyar.'